# Overview

TorchScript is a representation of a torch program. The torch program can be recreated from torch script and then run without Python dependency (like C++)

In [ ]:
import torch

`torch.jit.script` takes either an instance of `torch.nn.Module` or a function
can then returns a `ScriptModule` or `ScriptFunction`

# Modules

**Only methods marked with `@torch.jit.export`** will be **available** in the torch script model; to explicitly 

- `forward` is by default marked with `torch.jit.export`
- Exported codepath should not use non torch types

`@torch.jit.ignore` and `@torch.jit.unused` both **cause methods to not be converted** to torch script 

- `@torch.jit.ignore` **prevent** a model from being **exported**
- `@torch.jit.unused` **allows** a model to be **exported**, but the marked functions will **throw exceptions** if they are ever called

In [ ]:
class Model(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = torch.nn.Linear(3, 1)

    @torch.jit.export 
    def foo(self):
        pass 

    def forward(self, x):
        return self.linear(x)


model = Model()

model.eval()
model(torch.tensor([1, 2, 3], dtype=torch.float))

tensor([-1.0077], grad_fn=<AddBackward0>)

In [ ]:
torch_script = torch.jit.script(model)
torch_script
# torch_script.save("file")

RecursiveScriptModule(
  original_name=Model
  (linear): RecursiveScriptModule(original_name=Linear)
)

# Using Torch Script Modules

The created `ScriptModule` can be evaluated just as the original model or function

In [ ]:
torch_script.eval()
torch_script.forward(torch.tensor([1, 2, 3], dtype=torch.float))

tensor([-2.0935], grad_fn=<AddBackward0>)